#Assignment 1: Data Engineering
## 1. EXTRACT
### Step 1 : Load data with pandas

In [74]:
import pandas as pd

df = pd.read_csv("generations.csv")
df.head(10)


,id,timestamp,name,userId,input,output,latency,inputTokens,outputTokens,totalTokens,status,model,error
0,gen_001,2025-01-15T08:30:01Z,chat_completion,user_101,"{""messages"":[{""role"":""system"",""content"":""You a...","{""role"":""assistant"",""content"":""Hi! How can I h...",0.85,45,12,57,success,gpt-4o-mini,NaN
1,gen_002,2025-01-15T09:15:00Z,chat_completion,user_102,"{""messages"":[{""role"":""user"",""content"":""What is...","{""role"":""assistant"",""content"":""2+2 equals 4.""}",0.72,28,8,36,success,gpt-4o-mini,NaN
2,gen_003,2025-01-15T10:00:01Z,chat_completion,user_101,"{""messages"":[{""role"":""system"",""content"":""You a...","{""role"":""assistant"",""content"":""สวัสดี (sawatde...",1.24,52,15,67,success,gpt-4o,NaN
3,gen_004,2025-01-15T11:30:00Z,chat_completion,user_103,"{""messages"":[{""role"":""user"",""content"":""Write a...","{""role"":""assistant"",""content"":""Roses are red,\...",2.15,32,45,77,success,gpt-4o,NaN
4,gen_005,2025-01-15T12:00:00Z,chat_completion,user_101,"{""messages"":[{""role"":""user"",""content"":""Summari...",NaN,0.00,1250,0,1250,error,gpt-4o,context_length_exceeded
5,gen_006,2025-01-15T14:30:00Z,chat_completion,user_104,"{""messages"":[{""role"":""system"",""content"":""You a...","{""role"":""assistant"",""content"":""The function lo...",1.85,68,32,100,success,gpt-4o,NaN
6,gen_007,2025-01-15T15:45:00Z,chat_completion,user_102,"{""messages"":[{""role"":""user"",""content"":""Tell me...","{""role"":""assistant"",""content"":""Why did the pro...",0.95,25,18,43,success,gpt-4o-mini,NaN
7,gen_008,2025-01-15T16:20:00Z,chat_completion,user_105,"{""messages"":[{""role"":""user"",""content"":""Explain...","{""role"":""assistant"",""content"":""Quantum computi...",3.42,35,156,191,success,gpt-4o,NaN
8,gen_009,2025-01-15T17:00:00Z,chat_completion,user_103,"{""messages"":[{""role"":""user"",""content"":""Help me...",NaN,0.00,22,0,22,error,gpt-4o-mini,rate_limit_exceeded
9,gen_010,2025-01-15T18:30:00Z,chat_completion,user_106,"{""messages"":[{""role"":""system"",""content"":""You a...","{""role"":""assistant"",""content"":""SELECT * FROM u...",1.52,58,25,83,success,gpt-4o,NaN


###Step 2 : Data pre- processing

####2.1 check data information  understand what is the set of data were captured and the type of each column.  

- From the exucution demonstrates that the dataset has 50 records and 13 columns with multiple dtype( dtypes: float64(1), int64(3), object(9))


- From the data entity and background information provided, the records captured LArge Language Model (LLM) API calls across our applications.





In [75]:
#check data information
df.info()
print("Total rows:", len(df))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            50 non-null     object 
 1   timestamp     50 non-null     object 
 2   name          50 non-null     object 
 3   userId        50 non-null     object 
 4   input         50 non-null     object 
 5   output        43 non-null     object 
 6   latency       50 non-null     float64
 7   inputTokens   50 non-null     int64  
 8   outputTokens  50 non-null     int64  
 9   totalTokens   50 non-null     int64  
 10  status        50 non-null     object 
 11  model         50 non-null     object 
 12  error         7 non-null      object 
dtypes: float64(1), int64(3), object(9)
memory usage: 5.2+ KB
Total rows: 50


####2.2 checking NULL value in dataset

The output reflects that there is 7 records that assistent unable to return the outputs. Additionally, there is a negative corelation between output and errors. If thew errors occures, the output record will be 'NULL'  


In [76]:
# check NULL value
df.isnull().sum()

,0
id,0
timestamp,0
name,0
userId,0
input,0
output,7
latency,0
inputTokens,0
outputTokens,0
totalTokens,0


In [77]:
# observe the reasons once errors occured
df[df['error'].notna()]

,id,timestamp,name,userId,input,output,latency,inputTokens,outputTokens,totalTokens,status,model,error
4,gen_005,2025-01-15T12:00:00Z,chat_completion,user_101,"{""messages"":[{""role"":""user"",""content"":""Summari...",NaN,0.0,1250,0,1250,error,gpt-4o,context_length_exceeded
8,gen_009,2025-01-15T17:00:00Z,chat_completion,user_103,"{""messages"":[{""role"":""user"",""content"":""Help me...",NaN,0.0,22,0,22,error,gpt-4o-mini,rate_limit_exceeded
14,gen_015,2025-01-16T12:30:00Z,chat_completion,user_104,"{""messages"":[{""role"":""user"",""content"":""Generat...",NaN,0.0,25,0,25,error,gpt-4o,invalid_request
22,gen_023,2025-01-17T11:30:00Z,chat_completion,user_102,"{""messages"":[{""role"":""user"",""content"":""Fix thi...",NaN,0.0,18,0,18,error,gpt-4o-mini,content_filter
31,gen_032,2025-01-18T13:00:00Z,chat_completion,user_107,"{""messages"":[{""role"":""user"",""content"":""Generat...",NaN,0.0,22,0,22,error,gpt-4o,service_unavailable
40,gen_041,2025-01-19T14:30:00Z,chat_completion,user_108,"{""messages"":[{""role"":""user"",""content"":""Help""}]}",NaN,0.0,15,0,15,error,gpt-4o-mini,invalid_request
46,gen_047,2025-01-20T11:30:00Z,chat_completion,user_102,"{""messages"":[{""role"":""user"",""content"":""Debug t...",NaN,0.0,28,0,28,error,gpt-4o,rate_limit_exceeded


#2. Transform : Parse the input JSON to extract the user's message



###Step 1 : Create function to extract user's message from JSON

In [78]:
import json

def extract_user_message(input_str):
    try:
        # if empty
        if pd.isna(input_str) or input_str == "":
            return None
        # open Json
        data = json.loads(input_str)
        messages = data.get("messages", [])

        for msg in messages:
            if msg.get("role") == "user":
                return msg.get("content")

        return None

    except json.JSONDecodeError:
        return None

###Step 2 : Create column 'User message' and extract content into this column



In [79]:
#Create column user message and load content in
df["user_message"] = df["input"].apply(extract_user_message)


In [80]:
df.head(10)

,id,timestamp,name,userId,input,output,latency,inputTokens,outputTokens,totalTokens,status,model,error,user_message
0,gen_001,2025-01-15T08:30:01Z,chat_completion,user_101,"{""messages"":[{""role"":""system"",""content"":""You a...","{""role"":""assistant"",""content"":""Hi! How can I h...",0.85,45,12,57,success,gpt-4o-mini,NaN,Hello
1,gen_002,2025-01-15T09:15:00Z,chat_completion,user_102,"{""messages"":[{""role"":""user"",""content"":""What is...","{""role"":""assistant"",""content"":""2+2 equals 4.""}",0.72,28,8,36,success,gpt-4o-mini,NaN,What is 2+2?
2,gen_003,2025-01-15T10:00:01Z,chat_completion,user_101,"{""messages"":[{""role"":""system"",""content"":""You a...","{""role"":""assistant"",""content"":""สวัสดี (sawatde...",1.24,52,15,67,success,gpt-4o,NaN,Translate hello to Thai
3,gen_004,2025-01-15T11:30:00Z,chat_completion,user_103,"{""messages"":[{""role"":""user"",""content"":""Write a...","{""role"":""assistant"",""content"":""Roses are red,\...",2.15,32,45,77,success,gpt-4o,NaN,Write a poem
4,gen_005,2025-01-15T12:00:00Z,chat_completion,user_101,"{""messages"":[{""role"":""user"",""content"":""Summari...",NaN,0.00,1250,0,1250,error,gpt-4o,context_length_exceeded,Summarize this article...
5,gen_006,2025-01-15T14:30:00Z,chat_completion,user_104,"{""messages"":[{""role"":""system"",""content"":""You a...","{""role"":""assistant"",""content"":""The function lo...",1.85,68,32,100,success,gpt-4o,NaN,"Review this: function add(a,b){return a+b}"
6,gen_007,2025-01-15T15:45:00Z,chat_completion,user_102,"{""messages"":[{""role"":""user"",""content"":""Tell me...","{""role"":""assistant"",""content"":""Why did the pro...",0.95,25,18,43,success,gpt-4o-mini,NaN,Tell me a joke
7,gen_008,2025-01-15T16:20:00Z,chat_completion,user_105,"{""messages"":[{""role"":""user"",""content"":""Explain...","{""role"":""assistant"",""content"":""Quantum computi...",3.42,35,156,191,success,gpt-4o,NaN,Explain quantum computing
8,gen_009,2025-01-15T17:00:00Z,chat_completion,user_103,"{""messages"":[{""role"":""user"",""content"":""Help me...",NaN,0.00,22,0,22,error,gpt-4o-mini,rate_limit_exceeded,Help me debug
9,gen_010,2025-01-15T18:30:00Z,chat_completion,user_106,"{""messages"":[{""role"":""system"",""content"":""You a...","{""role"":""assistant"",""content"":""SELECT * FROM u...",1.52,58,25,83,success,gpt-4o,NaN,Write a query to get top 10 users


#3. Parse the output JSON to extract the assistant's response


In [81]:
def extract_assistant_response(output_str):
    if pd.isna(output_str) or output_str == "":
        return None

    try:
        data = json.loads(output_str)
        return data.get("content")
    except json.JSONDecodeError:
        return None

In [82]:
df["assistant_response"] = df["output"].apply(extract_assistant_response)


In [83]:
df.head(10)

,id,timestamp,name,userId,input,output,latency,inputTokens,outputTokens,totalTokens,status,model,error,user_message,assistant_response
0,gen_001,2025-01-15T08:30:01Z,chat_completion,user_101,"{""messages"":[{""role"":""system"",""content"":""You a...","{""role"":""assistant"",""content"":""Hi! How can I h...",0.85,45,12,57,success,gpt-4o-mini,NaN,Hello,Hi! How can I help you today?
1,gen_002,2025-01-15T09:15:00Z,chat_completion,user_102,"{""messages"":[{""role"":""user"",""content"":""What is...","{""role"":""assistant"",""content"":""2+2 equals 4.""}",0.72,28,8,36,success,gpt-4o-mini,NaN,What is 2+2?,2+2 equals 4.
2,gen_003,2025-01-15T10:00:01Z,chat_completion,user_101,"{""messages"":[{""role"":""system"",""content"":""You a...","{""role"":""assistant"",""content"":""สวัสดี (sawatde...",1.24,52,15,67,success,gpt-4o,NaN,Translate hello to Thai,สวัสดี (sawatdee)
3,gen_004,2025-01-15T11:30:00Z,chat_completion,user_103,"{""messages"":[{""role"":""user"",""content"":""Write a...","{""role"":""assistant"",""content"":""Roses are red,\...",2.15,32,45,77,success,gpt-4o,NaN,Write a poem,"Roses are red,\nViolets are blue..."
4,gen_005,2025-01-15T12:00:00Z,chat_completion,user_101,"{""messages"":[{""role"":""user"",""content"":""Summari...",NaN,0.00,1250,0,1250,error,gpt-4o,context_length_exceeded,Summarize this article...,None
5,gen_006,2025-01-15T14:30:00Z,chat_completion,user_104,"{""messages"":[{""role"":""system"",""content"":""You a...","{""role"":""assistant"",""content"":""The function lo...",1.85,68,32,100,success,gpt-4o,NaN,"Review this: function add(a,b){return a+b}",The function looks clean but could use type hi...
6,gen_007,2025-01-15T15:45:00Z,chat_completion,user_102,"{""messages"":[{""role"":""user"",""content"":""Tell me...","{""role"":""assistant"",""content"":""Why did the pro...",0.95,25,18,43,success,gpt-4o-mini,NaN,Tell me a joke,Why did the programmer quit? Because he didn't...
7,gen_008,2025-01-15T16:20:00Z,chat_completion,user_105,"{""messages"":[{""role"":""user"",""content"":""Explain...","{""role"":""assistant"",""content"":""Quantum computi...",3.42,35,156,191,success,gpt-4o,NaN,Explain quantum computing,Quantum computing uses quantum bits or qubits...
8,gen_009,2025-01-15T17:00:00Z,chat_completion,user_103,"{""messages"":[{""role"":""user"",""content"":""Help me...",NaN,0.00,22,0,22,error,gpt-4o-mini,rate_limit_exceeded,Help me debug,None
9,gen_010,2025-01-15T18:30:00Z,chat_completion,user_106,"{""messages"":[{""role"":""system"",""content"":""You a...","{""role"":""assistant"",""content"":""SELECT * FROM u...",1.52,58,25,83,success,gpt-4o,NaN,Write a query to get top 10 users,SELECT * FROM users ORDER BY created_at DESC L...


#3. Create a clean, flat table with these columns:
id, timestamp, user_id, model, status
user_message (extracted from input JSON)
assistant_response (extracted from output JSON)
latency, total_tokens, error

In [84]:
clean_df = df[
    [
        "id",
        "timestamp",
        "userId",
        "model",
        "status",
        "user_message",
        "assistant_response",
        "latency",
        "totalTokens",
        "error"
    ]
]


In [85]:
clean_df.head(10)

,id,timestamp,userId,model,status,user_message,assistant_response,latency,totalTokens,error
0,gen_001,2025-01-15T08:30:01Z,user_101,gpt-4o-mini,success,Hello,Hi! How can I help you today?,0.85,57,NaN
1,gen_002,2025-01-15T09:15:00Z,user_102,gpt-4o-mini,success,What is 2+2?,2+2 equals 4.,0.72,36,NaN
2,gen_003,2025-01-15T10:00:01Z,user_101,gpt-4o,success,Translate hello to Thai,สวัสดี (sawatdee),1.24,67,NaN
3,gen_004,2025-01-15T11:30:00Z,user_103,gpt-4o,success,Write a poem,"Roses are red,\nViolets are blue...",2.15,77,NaN
4,gen_005,2025-01-15T12:00:00Z,user_101,gpt-4o,error,Summarize this article...,None,0.00,1250,context_length_exceeded
5,gen_006,2025-01-15T14:30:00Z,user_104,gpt-4o,success,"Review this: function add(a,b){return a+b}",The function looks clean but could use type hi...,1.85,100,NaN
6,gen_007,2025-01-15T15:45:00Z,user_102,gpt-4o-mini,success,Tell me a joke,Why did the programmer quit? Because he didn't...,0.95,43,NaN
7,gen_008,2025-01-15T16:20:00Z,user_105,gpt-4o,success,Explain quantum computing,Quantum computing uses quantum bits or qubits...,3.42,191,NaN
8,gen_009,2025-01-15T17:00:00Z,user_103,gpt-4o-mini,error,Help me debug,None,0.00,22,rate_limit_exceeded
9,gen_010,2025-01-15T18:30:00Z,user_106,gpt-4o,success,Write a query to get top 10 users,SELECT * FROM users ORDER BY created_at DESC L...,1.52,83,NaN


- check malform rate to see the quality of data

In [86]:
clean_df["is_malformed"] = (
    clean_df["error"].notna() |
    clean_df["assistant_response"].isna() |
    clean_df["user_message"].isna()
)


/tmp/ipython-input-3498551084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df["is_malformed"] = (


In [87]:
malformed_rate = (clean_df["is_malformed"].sum() / len(clean_df)) * 100

print(f"Malformed rate: {malformed_rate:.2f}%")


Malformed rate: 14.00%


#4: Load cleaned_df into SQLlite and CSV file (For submit)

###Step 1 : Create database on SQLlite

In [88]:
import sqlite3

conn = sqlite3.connect("etl_assignment.db")

###Step 2 : Download table into database

In [89]:
clean_df.to_sql(
    "cleaned_generations",
    conn,
    if_exists="replace",
    index=False
)

50

###Step 3 : test queries

In [90]:
pd.read_sql("PRAGMA table_info(cleaned_generations)", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,id,TEXT,0,None,0
1,1,timestamp,TEXT,0,None,0
2,2,userId,TEXT,0,None,0
3,3,model,TEXT,0,None,0
4,4,status,TEXT,0,None,0
5,5,user_message,TEXT,0,None,0
6,6,assistant_response,TEXT,0,None,0
7,7,latency,REAL,0,None,0
8,8,totalTokens,INTEGER,0,None,0
9,9,error,TEXT,0,None,0


In [91]:
pd.read_sql("SELECT * FROM cleaned_generations LIMIT 5", conn)


,id,timestamp,userId,model,status,user_message,assistant_response,latency,totalTokens,error,is_malformed
0,gen_001,2025-01-15T08:30:01Z,user_101,gpt-4o-mini,success,Hello,Hi! How can I help you today?,0.85,57,None,0
1,gen_002,2025-01-15T09:15:00Z,user_102,gpt-4o-mini,success,What is 2+2?,2+2 equals 4.,0.72,36,None,0
2,gen_003,2025-01-15T10:00:01Z,user_101,gpt-4o,success,Translate hello to Thai,สวัสดี (sawatdee),1.24,67,None,0
3,gen_004,2025-01-15T11:30:00Z,user_103,gpt-4o,success,Write a poem,"Roses are red,\nViolets are blue...",2.15,77,None,0
4,gen_005,2025-01-15T12:00:00Z,user_101,gpt-4o,error,Summarize this article...,None,0.00,1250,context_length_exceeded,1


In [92]:
pd.read_sql("SELECT * FROM cleaned_generations LIMIT 5", conn)


,id,timestamp,userId,model,status,user_message,assistant_response,latency,totalTokens,error,is_malformed
0,gen_001,2025-01-15T08:30:01Z,user_101,gpt-4o-mini,success,Hello,Hi! How can I help you today?,0.85,57,None,0
1,gen_002,2025-01-15T09:15:00Z,user_102,gpt-4o-mini,success,What is 2+2?,2+2 equals 4.,0.72,36,None,0
2,gen_003,2025-01-15T10:00:01Z,user_101,gpt-4o,success,Translate hello to Thai,สวัสดี (sawatdee),1.24,67,None,0
3,gen_004,2025-01-15T11:30:00Z,user_103,gpt-4o,success,Write a poem,"Roses are red,\nViolets are blue...",2.15,77,None,0
4,gen_005,2025-01-15T12:00:00Z,user_101,gpt-4o,error,Summarize this article...,None,0.00,1250,context_length_exceeded,1


In [93]:
pd.read_sql("SELECT * FROM cleaned_generations WHERE error IS NOT NULL", conn)


,id,timestamp,userId,model,status,user_message,assistant_response,latency,totalTokens,error,is_malformed
0,gen_005,2025-01-15T12:00:00Z,user_101,gpt-4o,error,Summarize this article...,None,0.0,1250,context_length_exceeded,1
1,gen_009,2025-01-15T17:00:00Z,user_103,gpt-4o-mini,error,Help me debug,None,0.0,22,rate_limit_exceeded,1
2,gen_015,2025-01-16T12:30:00Z,user_104,gpt-4o,error,Generate test data,None,0.0,25,invalid_request,1
3,gen_023,2025-01-17T11:30:00Z,user_102,gpt-4o-mini,error,Fix this bug,None,0.0,18,content_filter,1
4,gen_032,2025-01-18T13:00:00Z,user_107,gpt-4o,error,Generate UUID,None,0.0,22,service_unavailable,1
5,gen_041,2025-01-19T14:30:00Z,user_108,gpt-4o-mini,error,Help,None,0.0,15,invalid_request,1
6,gen_047,2025-01-20T11:30:00Z,user_102,gpt-4o,error,Debug this code,None,0.0,28,rate_limit_exceeded,1


In [94]:
clean_df.to_csv("final_generations.csv", index=False)
